In [ ]:
import os
import http.client
import time
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil import tz

In [11]:
# 90-day Free Trial (start-date = 9/9/2018)
filename = 'api_key'

# using function from "Keeping API Keys Secret.ipynb"
# by https://github.com/dylburger
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

my_api_key = get_file_contents(filename)

In [13]:
basepath = 'data/id_files/'
end_str = '_game_ids_2017.csv'
paths = []
for fname in os.listdir(basepath):
    path = os.path.join(basepath, fname)
    if not os.path.isdir(path) and path.endswith(end_str):
        paths.append(path)
        
#paths

In [ ]:
#yr = 2016
yr = 2017

for path in paths:
    
    df = pd.read_csv(path)
    path = path.split(sep='/')
    path = path[2].split(sep='_')
    
    for index, row in df.iterrows():
        
        dt = row['date'].replace('-', '_')
        g_id = row['game_id'].replace('-', '_')
        
        out_file = "data/pbp_files/" + str(yr) + '/' + path[0] + '_' + path[1] + '/' + dt + '_' + row['team'] + '_vs_' + row['opp'] + '_' + g_id + '_pbp.json'
        #print(out_file)
        
        request = "/mlb/trial/v6.5/en/games/" + row['game_id'] + "/pbp.json?api_key=" + my_api_key
        #print(request)
        
        # send request to sportradar server using the HTTP request url.
        conn = http.client.HTTPSConnection("api.sportradar.us")
        conn.request("GET", request)
        
        # get response from server and store HTTPResponse instance
        res = conn.getresponse()
        
        # read response body and decode possible UTF-8 signature 
        # at the beginning of some files
        json_data = json.loads(res.read().decode("utf-8"))
        
        # write results to json output file
        with open(out_file, 'w') as outfile:
            json.dump(json_data, outfile)
        
        print("{} -- written".format(out_file))
        # Wait for 1 second
        time.sleep(1)

## END